# Train an object detection model using Tensorflow on SageMaker

## Setup environment

In [ ]:
import os
import sagemaker
from sagemaker.estimator import Framework, Estimator

role = sagemaker.get_execution_role()

inputs = {'train': '<your-data-s3-path>'} # define s3 training data inputs, this is the output of the processing job
tensorboard_s3_prefix = '<your-summaries-s3-path>' # s3 path for tensorboard events, up to you where to save events 

## Build and push container

In [ ]:
%%bash
git clone https://github.com/tensorflow/models.git docker/models
# get model_main and exporter_main files from TF2 Object Detection GitHub repository
cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
cp docker/models/research/object_detection/model_main_tf2.py source_dir

In [ ]:
image_name = 'tf2-object-detection'

In [ ]:
!sh ./docker/build_and_push.sh $image_name

In [ ]:
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

## Get pre-trained model from model zoo

Download the base model and extract locally

In [ ]:
%%bash
mkdir /tmp/checkpoint
mkdir source_dir/checkpoint
wget -O /tmp/efficientdet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
tar -zxvf /tmp/efficientdet.tar.gz --strip-components 2 --directory source_dir/checkpoint efficientdet_d1_coco17_tpu-32/checkpoint

## Create SageMaker Custom Framework and Launch Training job

Here we define a custom framework estimator using the Amazon SageMaker Python SDK and run training with that class, which will take care of managing these tasks.

In [ ]:
class CustomFramework(Framework):
    def __init__(
        self,
        entry_point,
        framework_version=None,
        py_version=None,
        source_dir=None,
        hyperparameters=None,
        image_uri=None,
        distribution=None,
        **kwargs
    ):
        super(CustomFramework, self).__init__(
            entry_point, source_dir, hyperparameters, image_uri=image_uri, **kwargs
        )
        self.framework_version = framework_version
        self.py_version = None
        
    def _configure_distribution(self, distributions):
        return None

    def create_model(
        self,
        model_server_workers=None,
        role=None,
        vpc_config_override=None,
        entry_point=None,
        source_dir=None,
        dependencies=None,
        image_uri=None,
        **kwargs
    ):
        return None

In [ ]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir":"/opt/training",        
        "pipeline_config_path": "pipeline.config",
        "num_train_steps": "1000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='ml.g5.2xlarge',
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection'
)

In [ ]:
estimator.fit(inputs)

## Visualize training metrics with Tensorboard

In [ ]:
#Due to this issue: https://github.com/ipython/ipykernel/issues/395#issuecomment-479787997
#If you're using a custom conda env, there is a change that the tensorboard executable isn't in the Python path.
#uncomment the following lines

#bin_env_path = "/home/ec2-user/anaconda3/envs/myenv/bin/"
#os.environ["PATH"] += os.pathsep + bin_env_path

In [ ]:
job_artifacts_path = estimator.latest_job_tensorboard_artifacts_path()
job_artifacts_path

### Visualize training outputs

In [ ]:
#Careful notebook would be stuck until you stop tensorboard, you can also launch this from a terminal
tensorboard_s3_output_path = f'{job_artifacts_path}/train'
!F_CPP_MIN_LOG_LEVEL=3 AWS_REGION=eu-west-1 tensorboard --logdir=$tensorboard_s3_output_path

### Visualize evaluation outputs


In [ ]:
#Careful notebook would be stuck until you stop tensorboard, you can also launch this from a terminal
tensorboard_s3_output_path = f'{job_artifacts_path}/eval'
!F_CPP_MIN_LOG_LEVEL=3 AWS_REGION=eu-west-1 tensorboard --logdir=$tensorboard_s3_output_path